# Rede Neural para Regressão Linear Múltipla
Este notebook implementa e treina uma rede neural simples para uma tarefa de regressão linear múltipla, utilizando uma arquitetura `ReLU → Identity`.

In [214]:
import numpy as np
from ann import initialize_layers, feed_forward
from backpropagation import backpropagation
from prepareData.prepareDataRegression import prepareDataRegression
from cost import cost_funcs
from training import train
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

## Configuração da Rede Neural
Aqui definimos a arquitetura da rede e a inicialização das camadas.

In [215]:
input_size = 7

layers = [
    {'neurons': 7, 'activation_function': 'relu'},
    {'neurons': 1, 'activation_function': 'identity'}
]

ann_layer = initialize_layers(layers, c_inputs=input_size)

## Preparação dos Dados
Utilizamos uma função auxiliar para carregar os dados de treino e teste.

In [216]:
x_train, x_test, y_train, y_test, min_max_values = prepareDataRegression()

print("Dados de entrada:")
print(x_train.columns.tolist())
print()

print("Dados de saída:")
print("House_Price")
print()

print("Shape dos dados:")
print(f"x_train: {x_train.shape}, y_train: {np.array(y_train).shape}")
print(f"x_test:  {x_test.shape},  y_test:  {np.array(y_test).shape}")

Dados de entrada:
['Square_Footage', 'Num_Bedrooms', 'Num_Bathrooms', 'Year_Built', 'Lot_Size', 'Garage_Size', 'Neighborhood_Quality']

Dados de saída:
House_Price

Shape dos dados:
x_train: (800, 7), y_train: (800,)
x_test:  (200, 7),  y_test:  (200,)


## Treinamento da Rede Neural
Treinamos a rede por 10 épocas utilizando a função de custo Mean Squared Error (`mse`) e uma taxa de aprendizado de `0.002`.

In [217]:
stages = train(
    ann=ann_layer,
    epochs=20,
    x=x_train,
    y=np.array(y_train),
    learning_rate=0.002,
    cost_func_name='mse'
)

## Avaliação do Modelo
Função para avaliar a acurácia utilizando o conjunto de testes e a função `regression_report`, criada usando as métricas de erro do `sklearn`.

In [218]:
def regression_report(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    report = (
        f"Regression Report\n"
        f"------------------------\n"

        f"Mean Squared Error (MSE):       {mse:.4f}\n"
        f"Root Mean Squared Error (RMSE): {locale.currency(rmse, grouping=True)}\n"
        f"Mean Absolute Error (MAE):      {mae:.4f}\n"
        f"R² Score:                       {r2:.4f}"
    )
    return report

def evaluate(ann, x, y, min_max_values):
  predictions = []
  for observation_id in range(len(x)):

    input = np.array(x.iloc[observation_id])
    prediction, _ = feed_forward(ann, input)
    predictions.append(prediction)

  scaler = MinMaxScaler(feature_range=min_max_values)
  scaler.fit(np.array([[0], [1]]))

  y_denormalized = scaler.transform(y.reshape(-1, 1))
  predictions_denormalized = scaler.transform(predictions)

  print("Resultados das Predições:")
  print(regression_report(y_denormalized, predictions_denormalized))

In [219]:
evaluate(stages, x_test, y_test, min_max_values)

Resultados das Predições:
Regression Report
------------------------
Mean Squared Error (MSE):       762960597.1761
Root Mean Squared Error (RMSE): R$ 27.621,74
Mean Absolute Error (MAE):      21829.6171
R² Score:                       0.9888


## Exemplos de Predições

In [225]:

for i in range(2):
  scaler = MinMaxScaler(feature_range=min_max_values)
  scaler.fit(np.array([[0], [1]]))
  input_vector = np.array(x_test.iloc[i])
  y_true = y_test[i]
  y_pred, _ = feed_forward(stages, input_vector)

  y_denormalized = scaler.transform(y_true.reshape(-1, 1))
  y_pred_denormalized = scaler.transform(y_pred.reshape(-1, 1))

  print(f"Exemplo {i+1}")
  print()
  print(f"Entrada (vetor de características):\n{np.round(input_vector, 3)}\n")
  print(f"Valor verdadeiro: R$ {(y_denormalized[0][0]):.2f}")
  print(f"Valor predito:    R$ {(y_pred_denormalized[0][0]):.2f}")
  print(f"Erro:             R$ {(y_denormalized[0][0] - y_pred_denormalized[0][0]):.2f}")
  print(f"Vetor de saída (identity): {np.round(y_pred, 3)}")
  print("\n")

Exemplo 1

Entrada (vetor de características):
[0.248 0.75  0.    0.486 0.946 1.    0.667]

Valor verdadeiro: R$ 418003.16
Valor predito:    R$ 402770.98
Erro:             R$ 15232.18
Vetor de saída (identity): [0.292]


Exemplo 2

Entrada (vetor de características):
[0.649 0.5   0.    0.139 0.907 0.    0.111]

Valor verdadeiro: R$ 731262.20
Valor predito:    R$ 762068.47
Erro:             R$ -30806.26
Vetor de saída (identity): [0.653]


